In [6]:
import json
import os
from PIL import Image
import pybboxes as pbx
import yaml
import shutil
import random
from collections import Counter, defaultdict
import cv2
from object_detection.src.data import create_filtered_coco_dataset, coco_to_yolo
from object_detection.src.data import merge_data_yamls, merge_datasets
from object_detection.src.analyzer import analyze_datasets

In [3]:
# Define base paths
original_base = "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco"
yolo_base = "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo"

dataset_types = ['train', 'val']  # Using 'train' and 'val' instead of 'train2017' and 'val2017'

# Create necessary directories within coco_filtered
for dataset_type in dataset_types:
    os.makedirs(os.path.join(yolo_base, dataset_type), exist_ok=True)

# Create annotations directory
os.makedirs(os.path.join(yolo_base, "annotations"), exist_ok=True)

# Paths for validation dataset
original_val_annotation = os.path.join(original_base, "annotations", "instances_val2017.json")
original_val_image_dir = "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco/val2017"  
filtered_val_annotation = os.path.join(yolo_base, "annotations", "instances_val2017.json")
filtered_val_image_dir = os.path.join(yolo_base, "val") 

In [4]:
# Create filtered validation dataset
create_filtered_coco_dataset(
    original_annotation_path=original_val_annotation,
    original_image_dir=original_val_image_dir,
    filtered_annotation_path=filtered_val_annotation,
    filtered_image_dir=filtered_val_image_dir
)

Filtered dataset created successfully at /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/val and /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/annotations/instances_val2017.json


In [5]:
# Paths for training dataset (assuming you have a similar filtered_train.json)
original_train_annotation = os.path.join(original_base, "annotations", "instances_train2017.json")
original_train_image_dir = "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco/train2017"  
filtered_train_annotation = os.path.join(yolo_base, "annotations", "instances_train2017.json")
filtered_train_image_dir = os.path.join(yolo_base, "train")  

In [6]:
create_filtered_coco_dataset(
    original_annotation_path=original_train_annotation,
    original_image_dir=original_train_image_dir,
    filtered_annotation_path=filtered_train_annotation,
    filtered_image_dir=filtered_train_image_dir
)

Filtered dataset created successfully at /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train and /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/annotations/instances_train2017.json


In [ ]:
# Paths for validation dataset
# yolo_base = "/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo"
# filtered_val_annotation = os.path.join(filtered_base, "annotations", "filtered_val.json")
# filtered_val_image_dir = os.path.join(filtered_base, "val") 

In [7]:
coco_to_yolo(
    coco_annotation_path=filtered_val_annotation,
    images_dir=filtered_val_image_dir,
    output_dir=os.path.join(yolo_base, "annotations", "yolo_format_val"),
    yaml_output_path= os.path.join(yolo_base, "annotations", "data_val.yaml"),
)

YOLO annotations and data.yaml have been saved to /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/annotations/yolo_format_val


In [8]:
coco_to_yolo(
    coco_annotation_path=filtered_train_annotation,
    images_dir=filtered_train_image_dir,
    output_dir=os.path.join(yolo_base, "annotations", "yolo_format_train"),
    yaml_output_path= os.path.join(yolo_base, "annotations", "data.yaml"),
)

Removed problematic image file: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train/000000022051.jpg
Removed problematic image file: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train/000000200365.jpg
Removed problematic image file: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train/000000183338.jpg
Removed problematic image file: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train/000000340038.jpg
Removed problematic image file: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train/000000550395.jpg
Removed problematic image file: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/train/000000077039.jpg
YOLO annotations and data.yaml have been saved to /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/annotations/yolo_format_t

# Add new data to coco

In [22]:
original_yamls = [
    '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/annotations/data.yaml',
    '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_forklift/annotations/data.yaml'
]

In [23]:
# Paths to the merged image directories
merged_train = '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift/train'
merged_val = '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift/val'
merged_test = '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift/test'

In [24]:
merged_yaml = '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift/data.yaml'
mappings_yaml = '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift/class_mappings.yaml'

In [25]:
os.makedirs(merged_train, exist_ok=True)
os.makedirs(merged_val, exist_ok=True)
os.makedirs(merged_test, exist_ok=True)

# Merge Yamls

In [26]:
merged_classes, class_mappings = merge_data_yamls(
    yaml_paths=original_yamls,
    merged_yaml_path=merged_yaml,
    merged_train_path=merged_train,
    merged_val_path=merged_val,
    merged_test_path=merged_test,  # Omit if not needed
    mappings_yaml_path=mappings_yaml  # Optional: save mappings to a YAML file
)


Merged Classes:
0: person
1: bicycle
2: car
3: motorcycle
4: airplane
5: bus
6: train
7: truck
8: boat
9: traffic light
10: fire hydrant
11: stop sign
12: parking meter
13: bench
14: bird
15: cat
16: dog
17: horse
18: sheep
19: cow
20: elephant
21: bear
22: zebra
23: giraffe
24: backpack
25: umbrella
26: handbag
27: tie
28: suitcase
29: frisbee
30: skis
31: snowboard
32: sports ball
33: kite
34: baseball bat
35: baseball glove
36: skateboard
37: surfboard
38: tennis racket
39: bottle
40: wine glass
41: cup
42: fork
43: knife
44: spoon
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
51: carrot
52: hot dog
53: pizza
54: donut
55: cake
56: chair
57: couch
58: potted plant
59: bed
60: dining table
61: toilet
62: tv
63: laptop
64: mouse
65: remote
66: keyboard
67: cell phone
68: microwave
69: oven
70: toaster
71: sink
72: refrigerator
73: book
74: clock
75: vase
76: scissors
77: teddy bear
78: hair drier
79: toothbrush
80: forklift

Class Mappings per Dataset:

Dataset: 

# Merge Data

In [27]:
# Define dataset parameters
dataset_1_params = {
    'data_dir': '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo',
    'train_images_dir': 'train/',
    'train_labels_dir': 'annotations/yolo_format_train',
    'val_images_dir': 'val/',
    'val_labels_dir': 'annotations/yolo_format_val',
    'test_images_dir': 'val/',
    'test_labels_dir': 'annotations/yolo_format_val',
}

dataset_2_params = {
    'data_dir': '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_forklift',
    'train_images_dir': 'train/',
    'train_labels_dir': 'annotations/yolo_format_train',
    'val_images_dir': 'val/',
    'val_labels_dir': 'annotations/yolo_format_val',
    'test_images_dir': 'test/',
    'test_labels_dir': 'annotations/yolo_format_test',
}

merge_datasets_list = [dataset_1_params, dataset_2_params]

# Define the path to class_mappings.yaml
mapping_yaml = '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift/class_mappings.yaml'

# Define merged dataset parameters
merge_dataset_params = {
    'data_dir': '/home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/merged_coco_forklift',
    'train_images_dir': 'train/images',
    'train_labels_dir': 'train/labels',
    'val_images_dir': 'val/images',
    'val_labels_dir': 'val/labels',
    'test_images_dir': 'test/images',
    'test_labels_dir': 'test/labels',
}

In [28]:
# Perform the merge
merge_datasets(
    merge_datasets=merge_datasets_list,
    mapping_yaml=mapping_yaml,
    merge_dataset_params=merge_dataset_params
)


--- Merging Completed ---
Total images to merge: 133055
Total labels to merge: 133055
Total images merged: 133055
Total labels merged: 133055
Verification Passed: All images and labels have been successfully merged.


In [29]:
# Perform the analysis
analyze_datasets(
    merge_datasets_list=merge_datasets_list,
    mapping_yaml=mapping_yaml,
    merge_dataset_params=merge_dataset_params
)


Analyzing Dataset: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo
  Using class mapping from: /home/LIBS/jozefov/projects/object_detection_avaltar/object_detection/data/coco_yolo/annotations/data.yaml
  Split: train
    Total Common Images and Labels: 118281
    Images without Labels: 0
    Labels without Images: 0
    Sample Label File: 000000441504.txt
      Original Label Content:
0 0.361446 0.532000 0.277108 0.216000
0 0.588855 0.218000 0.051205 0.140000
0 0.521084 0.206000 0.036145 0.084000
0 0.480422 0.199000 0.039157 0.086000
0 0.570783 0.513000 0.171687 0.214000
24 0.634036 0.570000 0.231928 0.208000
24 0.676205 0.795000 0.370482 0.222000
0 0.225904 0.184000 0.018072 0.032000
0 0.445783 0.194000 0.024096 0.060000
0 0.560241 0.193000 0.036145 0.078000
0 0.152108 0.188000 0.027108 0.048000
0 0.129518 0.192000 0.030120 0.044000
24 0.926205 0.746000 0.147590 0.240000
0 0.028614 0.207000 0.045181 0.038000
0 0.051205 0.199000 0.030120 0.050000
0